In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
import geopandas as gpd
import datetime as dt
import time

from shapely.geometry import MultiPolygon
from test_data_2d import clokes, journeys, example_route, example_space, islas
START, END = journeys['bc-tc']

In [11]:
import yaml
from pprint import pformat
with open("config.yml", "r") as cfg:
    ml_config = yaml.load(cfg, Loader=yaml.FullLoader)
    #print(type(config))
#s = pformat(config, indent=4, depth=12)
#print(yaml.dump(dict(config), indent=4))
print(ml_config)

{'dataset': {'barriers': 'clokes', 'origin': 'd', 'destination': 'f'}, 'logging': {'save_log': True, 'record_result': True, 'verbose': False}, 'visualisation': {'png': True, 'gif': False, 'short_gif': False, 'gif_zoom': 0.15, 'sol_txt': True}, 'validation': {'threshold': 1.5, 'no_intersect': True, 'invalidity_cost': 100, 'intersection_cost': 2}, 'defaults': {'ngen': 250, 'line_segments': 100, 'pop_size': 300, 'cxpb': 0.5, 'mutpb': 0.1, 'max_depth': 17, 'seed': 151, 'multiproc': 'imap', 'chunksize': 'None'}}


In [31]:
config = {}
config.update(list(ml_config.values()))
    
#l_config = {c.key():c.value() for c in ml_config.values()}
print(Namespace(**config))

ValueError: dictionary update sequence element #0 has length 3; 2 is required

In [30]:
ml_config.values()

dict_values([{'barriers': 'clokes', 'origin': 'd', 'destination': 'f'}, {'save_log': True, 'record_result': True, 'verbose': False}, {'png': True, 'gif': False, 'short_gif': False, 'gif_zoom': 0.15, 'sol_txt': True}, {'threshold': 1.5, 'no_intersect': True, 'invalidity_cost': 100, 'intersection_cost': 2}, {'ngen': 250, 'line_segments': 100, 'pop_size': 300, 'cxpb': 0.5, 'mutpb': 0.1, 'max_depth': 17, 'seed': 151, 'multiproc': 'imap', 'chunksize': 'None'}])

In [22]:
from argparse import Namespace
ns = Namespace()
ns.option = 'opt'
ns.var = 'vari'
ns.setting = 'set'
print(type(ns))
nsd = vars(ns)
print(type(nsd))
nsd.update(config)
#Namespace(**ns)
vars(ns)

<class 'argparse.Namespace'>


TypeError: 'NoneType' object is not callable

In [7]:
interval = (START.coords[0], END.coords[0])
interval = np.array([list(ele) for ele in list(interval)])
interval

array([[291107.6,  53789.1],
       [292653.3,  48962.3]])

In [8]:
np.array([[START.x, START.y], [END.x ,END.y]])

array([[291107.6,  53789.1],
       [292653.3,  48962.3]])